In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200828


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [7]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_新規Vintage")

#input
input_new = "NEW_CUST_Vintage"
input_all = "NEW_CUST_Vintage_all" #temporary用
input_all = "NEW_CUST_Vintage"

input_uri = "URI_Vintage.csv"
input_uri_all = "SUM_SHOPPING.csv"

#out_put
output = "NEW_CUST_Vintage_all"
result = "集計_新規顧客Vintage_by_channel"

output2 = "SUM_SHOPPING"
result2 = "集計_新規利用Vintage_by_channel"

result3 = "集計_新規Vintage_by_channel"

新規Vintage CUSTデータの読込＆集計

In [27]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv")
df = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')
#df = pd.concat([df1,df2])
df = df.sort_values(['cp','card','channel','mob'])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='utf-8-sig')

In [28]:
df = df[df['kanyu_yymm']>= 201803] #2018年3月以降に限定
df.head()

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,no_of_1stuse
0,20180331,2018,3,201803,ゴールド,オンライン,0,0,21,0,21,10
1,20180331,2018,3,201803,ゴールド,オンライン,0,1,4,0,4,2
2,20180331,2018,3,201803,ゴールド,店頭タブレット,0,0,171,0,171,41
3,20180331,2018,3,201803,ゴールド,店頭タブレット,0,1,50,0,50,12
4,20180331,2018,3,201803,ゴールド,郵送,0,0,169,0,169,89


In [29]:
#日付データの変換
df['cp']=pd.to_datetime(df['cp'], format='%Y%m%d')
df['kanyu_yymmdd'] = df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format='%Y%m%d')

df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month

df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,no_of_1stuse,kanyu_yymmdd,kanyu_year,kanyu_month
0,2018-03-31,2018,3,2018年3月,ゴールド,オンライン,0,0,21,0,21,10,2018-03-01,2018,3
1,2018-03-31,2018,3,2018年3月,ゴールド,オンライン,0,1,4,0,4,2,2018-03-01,2018,3
2,2018-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,0,171,0,171,41,2018-03-01,2018,3
3,2018-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,1,50,0,50,12,2018-03-01,2018,3
4,2018-03-31,2018,3,2018年3月,ゴールド,郵送,0,0,169,0,169,89,2018-03-01,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2020-06-30,2020,6,2018年5月,外商,郵送,25,1,0,0,1027,940,2018-05-01,2018,5
5303,2020-06-30,2020,6,2018年4月,外商,郵送,26,0,0,3,293,328,2018-04-01,2018,4
5304,2020-06-30,2020,6,2018年4月,外商,郵送,26,1,0,0,575,520,2018-04-01,2018,4
5289,2020-06-30,2020,6,2018年3月,外商,郵送,27,0,0,1,494,529,2018-03-01,2018,3


In [30]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(5669, 15)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            5345, 5346, 5331, 5332, 5317, 5318, 5303, 5304, 5289, 5290],
           dtype='int64', length=5669)
Index(['cp', 'year', 'month', 'kanyu_yymm', 'card', 'channel', 'mob', 'kzk_f',
       'no_of_new_account', 'no_of_cancel_account', 'no_of_account',
       'no_of_1stuse', 'kanyu_yymmdd', 'kanyu_year', 'kanyu_month'],
      dtype='object')
cp                      datetime64[ns]
year                             int64
month                            int64
kanyu_yymm                      object
card                            object
channel                         object
mob                              int64
kzk_f                            int64
no_of_new_account                int64
no_of_cancel_account             int64
no_of_account                    int64
no_of_1stuse                     int64
kanyu_yymmdd            datetime64[ns]
kanyu_year                       int64
kany

In [31]:
df['cp_date'] = df['cp'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_yymmdd'].dt.strftime('%Y/%m/%d')
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,no_of_new_account,no_of_cancel_account,no_of_account,no_of_1stuse,kanyu_yymmdd,kanyu_year,kanyu_month,cp_date,kanyu_date
0,2018-03-31,2018,3,2018年3月,ゴールド,オンライン,0,0,21,0,21,10,2018-03-01,2018,3,2018/03/31,2018/03/01
1,2018-03-31,2018,3,2018年3月,ゴールド,オンライン,0,1,4,0,4,2,2018-03-01,2018,3,2018/03/31,2018/03/01
2,2018-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,0,171,0,171,41,2018-03-01,2018,3,2018/03/31,2018/03/01
3,2018-03-31,2018,3,2018年3月,ゴールド,店頭タブレット,0,1,50,0,50,12,2018-03-01,2018,3,2018/03/31,2018/03/01
4,2018-03-31,2018,3,2018年3月,ゴールド,郵送,0,0,169,0,169,89,2018-03-01,2018,3,2018/03/31,2018/03/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2020-06-30,2020,6,2018年5月,外商,郵送,25,1,0,0,1027,940,2018-05-01,2018,5,2020/06/30,2018/05/01
5303,2020-06-30,2020,6,2018年4月,外商,郵送,26,0,0,3,293,328,2018-04-01,2018,4,2020/06/30,2018/04/01
5304,2020-06-30,2020,6,2018年4月,外商,郵送,26,1,0,0,575,520,2018-04-01,2018,4,2020/06/30,2018/04/01
5289,2020-06-30,2020,6,2018年3月,外商,郵送,27,0,0,1,494,529,2018-03-01,2018,3,2020/06/30,2018/03/01


In [32]:
#加入月別新規数集計
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,sum(no_of_new_account) as sum_new_account
        from 
            df
        where 
            mob  = 0
        and card not in ("ハウスゴールド")
        group by
            kanyu_yymm,card,channel
        """

df_new = sqldf(q, locals())
df_new

,kanyu_yymm,card,channel,sum_new_account
0,2018年10月,ゴールド,オンライン,32
1,2018年10月,ゴールド,店頭タブレット,223
2,2018年10月,ゴールド,郵送,207
3,2018年10月,一般,オンライン,126
4,2018年10月,一般,店頭タブレット,9234
...,...,...,...,...
191,2020年6月,ゴールド,郵送,82
192,2020年6月,一般,オンライン,665
193,2020年6月,一般,店頭タブレット,1027
194,2020年6月,一般,郵送,261


In [37]:
#加入月別口座数推移
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,mob
        ,sum(no_of_account) as sum_account
        ,sum(no_of_cancel_account) as sum_cancel
        ,sum(no_of_1stuse) as sum_1stuse
        from 
            df
        where 
            card not in ("ハウスゴールド")
        group by
            kanyu_yymm,card,channel,mob
        """

df_cust = sqldf(q, locals())
df_cust

,kanyu_yymm,card,channel,mob,sum_account,sum_cancel,sum_1stuse
0,2018年10月,ゴールド,オンライン,0,32,0,5
1,2018年10月,ゴールド,オンライン,1,30,2,25
2,2018年10月,ゴールド,オンライン,2,30,0,28
3,2018年10月,ゴールド,オンライン,3,30,0,29
4,2018年10月,ゴールド,オンライン,4,30,0,28
...,...,...,...,...,...,...,...
2837,2020年6月,ゴールド,郵送,0,81,1,42
2838,2020年6月,一般,オンライン,0,665,0,145
2839,2020年6月,一般,店頭タブレット,0,1024,3,181
2840,2020年6月,一般,郵送,0,261,0,75


In [42]:
#マージ
q = """
        SELECT
        t1.kanyu_yymm
        ,t1.card
        ,t1.channel
        ,t1.mob
        ,t1.sum_account
        ,t1.sum_cancel
        ,t1.sum_1stuse
        ,t2.sum_new_account
        from 
        df_cust t1
        left join 
        df_new t2
        on t1.kanyu_yymm = t2.kanyu_yymm
        and t1.card = t2.card
        and t1.channel = t2.channel
        order by 
        t1.kanyu_yymm,t1.card,t1.channel,t1.mob
        """

df_cust_shukei = sqldf(q, locals())
df_cust_shukei

,kanyu_yymm,card,channel,mob,sum_account,sum_cancel,sum_1stuse,sum_new_account
0,2018年10月,ゴールド,オンライン,0,32,0,5,32
1,2018年10月,ゴールド,オンライン,1,30,2,25,32
2,2018年10月,ゴールド,オンライン,2,30,0,28,32
3,2018年10月,ゴールド,オンライン,3,30,0,29,32
4,2018年10月,ゴールド,オンライン,4,30,0,28,32
...,...,...,...,...,...,...,...,...
2837,2020年6月,ゴールド,郵送,0,81,1,42,82
2838,2020年6月,一般,オンライン,0,665,0,145,665
2839,2020年6月,一般,店頭タブレット,0,1024,3,181,1027
2840,2020年6月,一般,郵送,0,261,0,75,261


In [44]:
#残存率と初回利用率
df_cust_shukei['survival_rate'] = df_cust_shukei['sum_account']/df_cust_shukei['sum_new_account']
df_cust_shukei['1st_use_rate'] = df_cust_shukei['sum_1stuse']/df_cust_shukei['sum_new_account']
df_cust_shukei.head()

,kanyu_yymm,card,channel,mob,sum_account,sum_cancel,sum_1stuse,sum_new_account,survival_rate,1st_use_rate
0,2018年10月,ゴールド,オンライン,0,32,0,5,32,1.0000,0.15625
1,2018年10月,ゴールド,オンライン,1,30,2,25,32,0.9375,0.78125
2,2018年10月,ゴールド,オンライン,2,30,0,28,32,0.9375,0.87500
3,2018年10月,ゴールド,オンライン,3,30,0,29,32,0.9375,0.90625
4,2018年10月,ゴールド,オンライン,4,30,0,28,32,0.9375,0.87500


In [45]:
#CSV出力
df_cust_shukei.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift-jis')

新規Vintage 売上データの読込＆集計

In [23]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_uri_all}",encoding='shift-jis')
df = pd.read_csv(data_dir /f"{input_uri_all}",encoding='shift-jis')
#df2 = pd.read_csv(data_dir /f"{input_uri}",encoding='shift-jis')
#df = pd.concat([df1,df2])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='utf-8-sig')

In [24]:
df = df[df['kanyu_yymm']>= 201803] #2018年3月以降に限定
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt
756,20190331,2019,3,201803,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1
757,20190331,2019,3,201803,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11
758,20190331,2019,3,201803,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4
759,20190331,2019,3,201803,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1
760,20190331,2019,3,201803,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,20200531,2020,5,202005,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1
28477,20200531,2020,5,202005,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19
28478,20200531,2020,5,202005,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5
28479,20200531,2020,5,202005,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36


In [25]:
#日付データの変換
df['cp']=pd.to_datetime(df['cp'], format='%Y%m%d')

df['kanyu_yymmdd'] = df['kanyu_yymm']*100+1
df['kanyu_yymmdd'] = pd.to_datetime(df['kanyu_yymmdd'], format='%Y%m%d')

df['kanyu_year'] = df['kanyu_yymmdd'].dt.year
df['kanyu_month'] = df['kanyu_yymmdd'].dt.month

df['kanyu_yymm'] = df[['kanyu_year', 'kanyu_month']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt,kanyu_yymmdd,kanyu_year,kanyu_month
756,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1,2018-03-01,2018,3
757,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11,2018-03-01,2018,3
758,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4,2018-03-01,2018,3
759,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1,2018-03-01,2018,3
760,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1,2018-03-01,2018,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,2020-05-31,2020,5,2020年5月,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1,2020-05-01,2020,5
28477,2020-05-31,2020,5,2020年5月,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19,2020-05-01,2020,5
28478,2020-05-31,2020,5,2020年5月,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5,2020-05-01,2020,5
28479,2020-05-31,2020,5,2020年5月,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36,2020-05-01,2020,5


In [26]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(9890, 18)
Int64Index([  756,   757,   758,   759,   760,   761,   762,   763,   764,
              765,
            ...
            28471, 28472, 28473, 28474, 28475, 28476, 28477, 28478, 28479,
            28480],
           dtype='int64', length=9890)
Index(['cp', 'year', 'month', 'kanyu_yymm', 'card', 'channel', 'mob', 'kzk_f',
       'hyakkaten_shoping_amt', 'gaibu_shoping_amt', 'shoping_amt',
       'hyakkaten_shopping_cnt', 'gaibu_shopping_cnt', 'shopping_cnt',
       'active_cnt', 'kanyu_yymmdd', 'kanyu_year', 'kanyu_month'],
      dtype='object')
cp                        datetime64[ns]
year                               int64
month                              int64
kanyu_yymm                        object
card                              object
channel                           object
mob                                int64
kzk_f                              int64
hyakkaten_shoping_amt              int64
gaibu_shoping_amt                float64
shoping_amt                 

In [27]:
df['cp_date'] = df['cp'].dt.strftime('%Y/%m/%d')
df['kanyu_date'] = df['kanyu_yymmdd'].dt.strftime('%Y/%m/%d')
df

,cp,year,month,kanyu_yymm,card,channel,mob,kzk_f,hyakkaten_shoping_amt,gaibu_shoping_amt,shoping_amt,hyakkaten_shopping_cnt,gaibu_shopping_cnt,shopping_cnt,active_cnt,kanyu_yymmdd,kanyu_year,kanyu_month,cp_date,kanyu_date
756,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,11,0,0,0.0,0,0,1,1,1,2018-03-01,2018,3,2019/03/31,2018/03/01
757,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,0,874299,461609.0,1335908,57,27,84,11,2018-03-01,2018,3,2019/03/31,2018/03/01
758,2019-03-31,2019,3,2018年3月,ゴールド,オンライン,12,1,157498,20242.0,177740,52,6,58,4,2018-03-01,2018,3,2019/03/31,2018/03/01
759,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,0,0,0.0,0,4,0,4,1,2018-03-01,2018,3,2019/03/31,2018/03/01
760,2019-03-31,2019,3,2018年3月,ゴールド,店頭タブレット,10,1,0,0.0,0,7,3,10,1,2018-03-01,2018,3,2019/03/31,2018/03/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28476,2020-05-31,2020,5,2020年5月,一般,店頭タブレット,0,1,182105,0.0,182105,2,0,2,1,2020-05-01,2020,5,2020/05/31,2020/05/01
28477,2020-05-31,2020,5,2020年5月,一般,郵送,0,0,166574,184652.0,351226,18,28,46,19,2020-05-01,2020,5,2020/05/31,2020/05/01
28478,2020-05-31,2020,5,2020年5月,一般,郵送,0,1,5670,106692.0,112362,7,19,26,5,2020-05-01,2020,5,2020/05/31,2020/05/01
28479,2020-05-31,2020,5,2020年5月,外商,郵送,0,0,1832272,749391.0,2581663,71,110,181,36,2020-05-01,2020,5,2020/05/31,2020/05/01


In [28]:
#新規数再確認
df_new

,kanyu_yymm,card,channel,sum_new_account
0,2018年10月,ゴールド,オンライン,32
1,2018年10月,ゴールド,店頭タブレット,223
2,2018年10月,ゴールド,郵送,207
3,2018年10月,一般,オンライン,126
4,2018年10月,一般,店頭タブレット,9234
...,...,...,...,...
184,2020年5月,ゴールド,郵送,57
185,2020年5月,一般,オンライン,565
186,2020年5月,一般,店頭タブレット,243
187,2020年5月,一般,郵送,134


#加入月別新規数集計
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,sum(no_of_new_account) as sum_new_account
        from 
        df
        where 
        mob  = 0
        group by
        kanyu_yymm,card,channel
        """

df_new = sqldf(q, locals())
df_new

In [29]:
#加入月別売上実績
q = """
        SELECT
        kanyu_yymm
        ,card
        ,channel
        ,mob
        ,sum(hyakkaten_shoping_amt) as sum_hyakkaten_amt
        ,sum(gaibu_shoping_amt) as sum_gaibu_amt
        ,sum(shoping_amt) as sum_shopping_amt
        ,sum(hyakkaten_shopping_cnt) as sum_hyakkaten_trn
        ,sum(gaibu_shopping_cnt) as sum_gaibu_trn
        ,sum(shopping_cnt) as sum_shopping_trn
        ,sum(active_cnt) as sum_act_cnt
        from 
        df
        where 
        card not in ("ハウスゴールド")
        group by
        kanyu_yymm,card,channel,mob
        """

df_uri = sqldf(q, locals())
df_uri.head()

,kanyu_yymm,card,channel,mob,sum_hyakkaten_amt,sum_gaibu_amt,sum_shopping_amt,sum_hyakkaten_trn,sum_gaibu_trn,sum_shopping_trn,sum_act_cnt
0,2018年10月,ゴールド,オンライン,0,212068,135812.0,347880,13,16,29,5
1,2018年10月,ゴールド,オンライン,1,6601935,2878117.0,9480052,79,50,129,22
2,2018年10月,ゴールド,オンライン,2,3097533,355472.0,3453005,82,31,113,17
3,2018年10月,ゴールド,オンライン,3,4327568,501915.0,4829483,54,50,104,17
4,2018年10月,ゴールド,オンライン,4,842844,1127599.0,1970443,33,41,74,15


In [30]:
#マージ
q = """
        SELECT
        t1.kanyu_yymm
        ,t1.card
        ,t1.channel
        ,t1.mob
        ,t1.sum_hyakkaten_amt
        ,sum(t1.sum_hyakkaten_amt) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_hyakkaten_amt
        ,t1.sum_gaibu_amt
        ,sum(t1.sum_gaibu_amt) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_gaibu_amt
        ,t1.sum_shopping_amt
        ,sum(t1.sum_shopping_amt) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_shopping_amt
        ,t1.sum_hyakkaten_trn
        ,sum(t1.sum_hyakkaten_trn) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_hyakkaten_trn
        ,t1.sum_gaibu_trn
        ,sum(t1.sum_gaibu_trn) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_gaibu_trn
        ,t1.sum_shopping_trn
        ,sum(t1.sum_shopping_trn) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_shopping_trn
        ,t1.sum_act_cnt
        ,sum(t1.sum_act_cnt) over(partition by t1.kanyu_yymm,t1.card,t1.channel order by mob) as cum_act_cnt
        ,t2.sum_new_account
        from 
        df_uri t1
        left join 
        df_new t2
        on t1.kanyu_yymm = t2.kanyu_yymm
        and t1.card = t2.card
        and t1.channel = t2.channel
        order by 
        t1.kanyu_yymm,t1.card,t1.channel,t1.mob
        """

df_uri_shukei = sqldf(q, locals())
df_uri_shukei

,kanyu_yymm,card,channel,mob,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account
0,2018年10月,ゴールド,オンライン,0,212068,212068,135812.0,135812.0,347880,347880,13,13,16,16,29,29,5,5,32
1,2018年10月,ゴールド,オンライン,1,6601935,6814003,2878117.0,3013929.0,9480052,9827932,79,92,50,66,129,158,22,27,32
2,2018年10月,ゴールド,オンライン,2,3097533,9911536,355472.0,3369401.0,3453005,13280937,82,174,31,97,113,271,17,44,32
3,2018年10月,ゴールド,オンライン,3,4327568,14239104,501915.0,3871316.0,4829483,18110420,54,228,50,147,104,375,17,61,32
4,2018年10月,ゴールド,オンライン,4,842844,15081948,1127599.0,4998915.0,1970443,20080863,33,261,41,188,74,449,15,76,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,2020年5月,ゴールド,郵送,0,1313049,1313049,2587186.0,2587186.0,3900235,3900235,26,26,262,262,288,288,28,28,57
2642,2020年5月,一般,オンライン,0,1645253,1645253,2399509.0,2399509.0,4044762,4044762,40,40,612,612,652,652,187,187,565
2643,2020年5月,一般,店頭タブレット,0,630130,630130,365238.0,365238.0,995368,995368,49,49,61,61,110,110,30,30,243
2644,2020年5月,一般,郵送,0,172244,172244,291344.0,291344.0,463588,463588,25,25,47,47,72,72,24,24,134


In [31]:
df_uri_shukei['百貨店売上vin'] = df_uri_shukei['cum_hyakkaten_amt']/df_uri_shukei['sum_new_account']#百貨店売上Vin
df_uri_shukei['外部売上vin'] = df_uri_shukei['cum_gaibu_amt']/df_uri_shukei['sum_new_account']#外部売上Vin
df_uri_shukei['合計売上vin'] = df_uri_shukei['cum_shopping_amt']/df_uri_shukei['sum_new_account']#合計売上Vin

df_uri_shukei

,kanyu_yymm,card,channel,mob,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account,百貨店売上vin,外部売上vin,合計売上vin
0,2018年10月,ゴールド,オンライン,0,212068,212068,135812.0,135812.0,347880,347880,13,13,16,16,29,29,5,5,32,6627.125000,4244.125000,10871.250000
1,2018年10月,ゴールド,オンライン,1,6601935,6814003,2878117.0,3013929.0,9480052,9827932,79,92,50,66,129,158,22,27,32,212937.593750,94185.281250,307122.875000
2,2018年10月,ゴールド,オンライン,2,3097533,9911536,355472.0,3369401.0,3453005,13280937,82,174,31,97,113,271,17,44,32,309735.500000,105293.781250,415029.281250
3,2018年10月,ゴールド,オンライン,3,4327568,14239104,501915.0,3871316.0,4829483,18110420,54,228,50,147,104,375,17,61,32,444972.000000,120978.625000,565950.625000
4,2018年10月,ゴールド,オンライン,4,842844,15081948,1127599.0,4998915.0,1970443,20080863,33,261,41,188,74,449,15,76,32,471310.875000,156216.093750,627526.968750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,2020年5月,ゴールド,郵送,0,1313049,1313049,2587186.0,2587186.0,3900235,3900235,26,26,262,262,288,288,28,28,57,23035.947368,45389.228070,68425.175439
2642,2020年5月,一般,オンライン,0,1645253,1645253,2399509.0,2399509.0,4044762,4044762,40,40,612,612,652,652,187,187,565,2911.952212,4246.918584,7158.870796
2643,2020年5月,一般,店頭タブレット,0,630130,630130,365238.0,365238.0,995368,995368,49,49,61,61,110,110,30,30,243,2593.127572,1503.037037,4096.164609
2644,2020年5月,一般,郵送,0,172244,172244,291344.0,291344.0,463588,463588,25,25,47,47,72,72,24,24,134,1285.402985,2174.208955,3459.611940


In [32]:
df_uri_shukei = df_uri_shukei.sort_values(['kanyu_yymm','card','channel','mob'])

In [33]:
#CSV出力
df_uri_shukei.to_csv(data_dir /f'{result2}.csv', header=True ,index=None, encoding='utf-8-sig')

In [35]:
df_merge = pd.merge(df_cust_shukei,df_uri_shukei, on =['kanyu_yymm','card','channel','mob'], how = 'left')
df_merge

,kanyu_yymm,card,channel,mob,sum_account,sum_kaiyaku,sum_new_account_x,残存率,sum_hyakkaten_amt,cum_hyakkaten_amt,sum_gaibu_amt,cum_gaibu_amt,sum_shopping_amt,cum_shopping_amt,sum_hyakkaten_trn,cum_hyakkaten_trn,sum_gaibu_trn,cum_gaibu_trn,sum_shopping_trn,cum_shopping_trn,sum_act_cnt,cum_act_cnt,sum_new_account_y,百貨店売上vin,外部売上vin,合計売上vin
0,2018年10月,ゴールド,オンライン,0,32,NaN,32,1.00000,212068,212068,135812.0,135812.0,347880,347880,13,13,16,16,29,29,5,5,32,6627.125000,4244.125000,10871.250000
1,2018年10月,ゴールド,オンライン,1,30,NaN,32,0.93750,6601935,6814003,2878117.0,3013929.0,9480052,9827932,79,92,50,66,129,158,22,27,32,212937.593750,94185.281250,307122.875000
2,2018年10月,ゴールド,オンライン,2,30,NaN,32,0.93750,3097533,9911536,355472.0,3369401.0,3453005,13280937,82,174,31,97,113,271,17,44,32,309735.500000,105293.781250,415029.281250
3,2018年10月,ゴールド,オンライン,3,30,NaN,32,0.93750,4327568,14239104,501915.0,3871316.0,4829483,18110420,54,228,50,147,104,375,17,61,32,444972.000000,120978.625000,565950.625000
4,2018年10月,ゴールド,オンライン,4,30,NaN,32,0.93750,842844,15081948,1127599.0,4998915.0,1970443,20080863,33,261,41,188,74,449,15,76,32,471310.875000,156216.093750,627526.968750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,2020年5月,ゴールド,郵送,0,57,0.0,57,1.00000,1313049,1313049,2587186.0,2587186.0,3900235,3900235,26,26,262,262,288,288,28,28,57,23035.947368,45389.228070,68425.175439
2642,2020年5月,一般,オンライン,0,564,1.0,565,0.99823,1645253,1645253,2399509.0,2399509.0,4044762,4044762,40,40,612,612,652,652,187,187,565,2911.952212,4246.918584,7158.870796
2643,2020年5月,一般,店頭タブレット,0,241,2.0,243,0.99177,630130,630130,365238.0,365238.0,995368,995368,49,49,61,61,110,110,30,30,243,2593.127572,1503.037037,4096.164609
2644,2020年5月,一般,郵送,0,134,0.0,134,1.00000,172244,172244,291344.0,291344.0,463588,463588,25,25,47,47,72,72,24,24,134,1285.402985,2174.208955,3459.611940


In [36]:
#CSV出力
df_merge.to_csv(data_dir /f'{result3}.csv', header=True ,index=None, encoding='utf-8-sig')

In [44]:
#df_uri_shukei.to_excel(data_dir /f'{result2}_test.xlsx', header=True, index=None,sheet_name='new_sheet_name')